In [34]:
from sage.all import Polyhedron

# Check if a convex set A is contained in convex set B
def is_subset(convex_set_A, convex_set_B):
    poly_B = Polyhedron(vertices=convex_set_B)
    
    # Check if each point in convex_set_A is inside poly_B
    for point in convex_set_A:
        if not poly_B.contains(vector(point)):
            return False
    return True

# Find the largest polyhedron that contains all other polyhedra in a given set of polyhedra
def find_largest_polyhedron(polyhedra):
    for candidate in polyhedra:
        if all(is_subset(poly.vertices_list(), candidate.vertices_list()) for poly in polyhedra if poly != candidate):
            return candidate  # Return the largest polyhedron that contains all others
    # If no polyhedron contains all others, output an error message
    print("Error: No subset relationship among polyhedra.")
    return None

# Replace the union of intersections of polyhedras with the quadrants with the largest polyhedron if it exists
def compute_union_polyhedron(H_lists, quadrants):
    union_polyhedra = []
    for quadrant in quadrants:
        quadrant_unions = []
        for H_list in H_lists:
            polyhedra = []
            for ieq in H_list:
                H = Polyhedron(ieqs=[ieq])
                intersection = H.intersection(quadrant)
                polyhedra.append(intersection)
            # Find the largest polyhedron that contains all others, if not found, return error
            largest_polyhedron = find_largest_polyhedron(polyhedra)
            if largest_polyhedron is None:
                return None
            quadrant_unions.append(largest_polyhedron)
        union_polyhedra.append(quadrant_unions)
    return union_polyhedra

# 13 B_1^3
Ball = Polyhedron(vertices=[[0, 0, 13], [13, 0, 0], [0, 13, 0], [0, 0, -13], [0, -13, 0], [-13, 0, 0]])

# Define the polyhedra for the eight quadrants
quadrants = [
    Polyhedron(ieqs=[(0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1)]).intersection(Ball),    # first quadrant
    Polyhedron(ieqs=[(0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, -1)]).intersection(Ball),   # second quadrant
    Polyhedron(ieqs=[(0, 1, 0, 0), (0, 0, -1, 0), (0, 0, 0, 1)]).intersection(Ball),   # third quadrant
    Polyhedron(ieqs=[(0, -1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1)]).intersection(Ball),   # fourth quadrant
    Polyhedron(ieqs=[(0, 1, 0, 0), (0, 0, -1, 0), (0, 0, 0, -1)]).intersection(Ball),  # fifth quadrant
    Polyhedron(ieqs=[(0, -1, 0, 0), (0, 0, 1, 0), (0, 0, 0, -1)]).intersection(Ball),  # sixth quadrant
    Polyhedron(ieqs=[(0, -1, 0, 0), (0, 0, -1, 0), (0, 0, 0, 1)]).intersection(Ball),  # seventh quadrant
    Polyhedron(ieqs=[(0, -1, 0, 0), (0, 0, -1, 0), (0, 0, 0, -1)]).intersection(Ball)  # eighth quadrant
]

# Define the list of constraints such that x not in Ki, for each x in R^3.
H_lists = [
    [(-1, -1, -1, -1), (-1, -1, 1, 1), (-1, -1, -1, 1), (-1, -1, 1, -1)],
    [(-1, -1, -1, -1), (-1, 1, -1, 1), (-1, 1, -1, -1), (-1, -1, -1, 1)],
    [(-1, -1, -1, -1), (-1, 1, 1, -1), (-1, 1, -1, -1), (-1, -1, 1, -1)],
    [(-1, 1, 1, 1), (-1, 1, 1, -1), (-1, 1, -1, 1), (-1, 1, -1, -1)],
    [(-1, 1, 1, 1), (-1, -1, 1, 1), (-1, 1, 1, -1), (-1, -1, 1, -1)],
    [(-1, 1, 1, 1), (-1, -1, 1, 1), (-1, 1, -1, 1), (-1, -1, -1, 1)]
]

# Compute (Ok \cap 13B_1^3)\setminus Ki)
union_polyhedra = compute_union_polyhedron(H_lists, quadrants)

# Compute \bigcap_{i\in[3]} ((Ok \cap 13B_1^3)\setminus Ki) \cap \bigcap_{i\in[3]} ((Ok \cap 13B_1^3)\setminus (-Ki))
if union_polyhedra is not None:
    S = []
    for q_index, quadrant_unions in enumerate(union_polyhedra):
        Si = quadrant_unions[0]
        for h_index in range(1, len(quadrant_unions)):
            Si = Si.intersection(quadrant_unions[h_index])
        S.append(Si)

    # Print and plot the vertices of S1 to S8
    for idx, Si in enumerate(S, start=1):
        print(f"Vertices of S{idx} (intersection of union_polyhedra):", Si.vertices_list())
        """
        p_Si = Si.plot()
        for v in Si.vertices_list():
            p_Si += text3d(f'{v}', v, fontsize=20, color='red')
        p_Si.show()
        """
else:
    print("Computation terminated due to no subset relationship among polyhedra.")



Vertices of S1 (intersection of union_polyhedra): [[6, 6, 1], [1, 1, 1], [1, 6, 6], [6, 1, 6]]
Vertices of S2 (intersection of union_polyhedra): [[6, 6, -1], [1, 1, -1], [1, 6, -6], [6, 1, -6]]
Vertices of S3 (intersection of union_polyhedra): [[6, -1, 6], [1, -1, 1], [1, -6, 6], [6, -6, 1]]
Vertices of S4 (intersection of union_polyhedra): [[-1, 6, 6], [-1, 1, 1], [-6, 6, 1], [-6, 1, 6]]
Vertices of S5 (intersection of union_polyhedra): [[6, -1, -6], [1, -1, -1], [1, -6, -6], [6, -6, -1]]
Vertices of S6 (intersection of union_polyhedra): [[-1, 6, -6], [-1, 1, -1], [-6, 6, -1], [-6, 1, -6]]
Vertices of S7 (intersection of union_polyhedra): [[-1, -1, 1], [-6, -6, 1], [-6, -1, 6], [-1, -6, 6]]
Vertices of S8 (intersection of union_polyhedra): [[-1, -1, -1], [-6, -6, -1], [-6, -1, -6], [-1, -6, -6]]


In [32]:
S1 = Polyhedron(vertices=[[6, 1, 6], [1, 6, 6], [6, 6, 1], [1, 1, 1]])
S2 = Polyhedron(vertices=[[6, 1, -6], [1, 6, -6], [6, 6, -1], [1, 1, -1]])
S5 = Polyhedron(vertices=[[6, -1, -6], [1, -6, -6], [6, -6, -1], [1, -1, -1]])
S6 = Polyhedron(vertices=[[-6, 1, -6], [-1, 6, -6], [-6, 6, -1], [-1, 1, -1]])

# Define halfspaces and intersections
H11 = Polyhedron(ieqs=[(-7, 1, 1, 1)])
H12 = Polyhedron(ieqs=[(7, -1, -1, -1)])
P11 = S1.intersection(H11)
P12 = S1.intersection(H12)

# EXT (S1.intersection(H12))
for q in P12.Vrepresentation():
    print(q)

H51 = Polyhedron(ieqs=[(-7, 1, -1, -1)])
H52 = Polyhedron(ieqs=[(7, -1, 1, 1)])
P51 = S5.intersection(H51)
P52 = S5.intersection(H52)

H61 = Polyhedron(ieqs=[(-7, -1, 1, -1)])
H62 = Polyhedron(ieqs=[(7, 1, -1, 1)])
P61 = S6.intersection(H61)
P62 = S6.intersection(H62)

P2 = Polyhedron(ieqs=[(9, -1, -1, -1), (9, 1, -1, 1), (9, -1, 1, 1)])
P21 = S2.intersection(P2)
P22 = S2.intersection(Polyhedron(ieqs=[(-9, 1, 1, 1)]))
P23 = S2.intersection(Polyhedron(ieqs=[(-9, 1, -1, -1)]))
P24 = S2.intersection(Polyhedron(ieqs=[(-9, -1, 1, -1)]))

# Calculate the maximum 1-norm from given points to each convex hull's vertices
def calculate_diameter(intersection, a): 
    vertices = intersection.vertices_list() 
    max_distance = 0 
    for i in range(len(vertices)): 
        distance = sum(abs(vertices[i][k] - a[k]) for k in range(3)) 
        if distance > max_distance: 
            max_distance = distance 
    return max_distance

a = [5, 5, 4]
b = [5, -4, -5]
c = [-4, 5, -5]
d = [2, 2, -2]
threshold = 7

# Print results
if calculate_diameter(P11, a) <= threshold:
    print("\\EXT P11 \\subseteq 7B13+a.")
if calculate_diameter(P22, a) <= threshold:
    print("\\EXT P22 \\subseteq 7B13+a.")
if calculate_diameter(P51, b) <= threshold:
    print("\\EXT P51 \\subseteq 7B13+b.")
if calculate_diameter(P23, b) <= threshold:
    print("\\EXT P23 \\subseteq 7B13+b.")
if calculate_diameter(P61, c) <= threshold:
    print("\\EXT P61 \\subseteq 7B13+c.")
if calculate_diameter(P24, c) <= threshold:
    print("\\EXT P24 \\subseteq 7B13+c.")
if calculate_diameter(P12, d) <= threshold:
    print("\\EXT P12 \\subseteq 7B13+d.")
if calculate_diameter(P52, d) <= threshold:
    print("\\EXT P52 \\subseteq 7B13+d.")
if calculate_diameter(P62, d) <= threshold:
    print("\\EXT P62 \\subseteq 7B13+d.")
if calculate_diameter(P21, d) <= threshold:
    print("\\EXT P21 \\subseteq 7B13+d.")


A vertex at (3, 3, 1)
A vertex at (3, 1, 3)
A vertex at (1, 1, 1)
A vertex at (1, 3, 3)
\EXT P11 \subseteq 7B13+a.
\EXT P22 \subseteq 7B13+a.
\EXT P51 \subseteq 7B13+b.
\EXT P23 \subseteq 7B13+b.
\EXT P61 \subseteq 7B13+c.
\EXT P24 \subseteq 7B13+c.
\EXT P12 \subseteq 7B13+d.
\EXT P52 \subseteq 7B13+d.
\EXT P62 \subseteq 7B13+d.
\EXT P21 \subseteq 7B13+d.


In [31]:
from sage.all import Polyhedron

# Define eight polytopes
T1 = Polyhedron(vertices=[(1, 6, 6), (6, 1, 6), (6, 6, 1)])
T2 = Polyhedron(vertices=[(1, 6, -6), (6, 1, -6), (6, 6, -1)])
T3 = Polyhedron(vertices=[(1, -6, 6), (6, -1, 6), (6, -6, 1)])
T4 = Polyhedron(vertices=[(-1, 6, 6), (-6, 1, 6), (-6, 6, 1)])
T5 = Polyhedron(vertices=[(1, -6, -6), (6, -1, -6), (6, -6, -1)])
T6 = Polyhedron(vertices=[(-1, 6, -6), (-6, 1, -6), (-6, 6, -1)])
T7 = Polyhedron(vertices=[(-1, -6, 6), (-6, -1, 6), (-6, -6, 1)])
T8 = Polyhedron(vertices=[(-1, -6, -6), (-6, -1, -6), (-6, -6, -1)])

Tk_list = [T1, T2, T3, T4, T5, T6, T7, T8]

# Define inequalities for H_{f_4}^3+ and H_{-f_3}^3+
H1 = Polyhedron(ieqs=[[-3, -1, 1, 1]])
H2 = Polyhedron(ieqs=[[-3, -1, 1, -1]])

# Calculate intersections
T1_H1 = T1.intersection(H1)
T2_H2 = T2.intersection(H2)

# Print the vertices of the intersections
print(f"Vertices of T1 ∩ H1: {T1_H1.vertices_list()}")
print(f"Vertices of T2 ∩ H2: {T2_H2.vertices_list()}")

# Other points and calculations
A = [
    (6, -1, 6), (-1, 6, 6), (4, -3, -6), (-3, 4, -6), (-2, -6, 5), (-6, -4, -3), (6, 5, -2)
]

A_prime = [
    (6, -1, 6), (-1, 6, 6), (4, -3, -6), (-3, 4, -6), (-6, -2, 5), (-4, -6, -3), (5, 6, -2)
]

P = [ 
    (-6, -4, -3), (-3, -4, -6), (5, -6, -2), (-5, 6, -2), (6, -5, 2)
]

# Determine which Tk a point belongs to
def find_Tk(point, Tk_list):
    for idx, T in enumerate(Tk_list, 1):
        if T.contains(point):
            return idx
    return None

# Calculate the minimum 1-norm between elements in a point set
def min_distance(points):
    min_dist = float('inf')
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            dist = sum(abs(points[i][k] - points[j][k]) for k in range(3))
            if dist < min_dist:
                min_dist = dist
    return min_dist

# Print results
for idx, point in enumerate(A, 1):
    Tk_index = find_Tk(point, Tk_list)
    if Tk_index:
        print(f"Point {point} in A belongs to T{Tk_index}.")
    else:
        print(f"Point {point} in A does not belong to any Tk.")

for idx, point in enumerate(A_prime, 1):
    Tk_index = find_Tk(point, Tk_list)
    if Tk_index:
        print(f"Point {point} in A' belongs to T{Tk_index}.")
    else:
        print(f"Point {point} in A' does not belong to any Tk.")
for idx, point in enumerate(P, 1):
    Tk_index = find_Tk(point, Tk_list)
    if Tk_index:
        print(f"Point {point} in P belongs to T{Tk_index}.")
    else:
        print(f"Point {point} in P does not belong to any Tk.")

# Calculate and print the minimum 1-norm between elements in A
min_dist_A = min_distance(A)
if min_dist_A >= 14:
    print("A, 14-separated")

# Calculate and print the minimum 1-norm between elements in A'
min_dist_A_prime = min_distance(A_prime)
if min_dist_A_prime >= 14:
    print("A', 14-separated")



Vertices of T1 ∩ H1: [[5, 2, 6], [5, 6, 2], [1, 6, 6]]
Vertices of T2 ∩ H2: [[5, 2, -6], [5, 6, -2], [1, 6, -6]]
Point (6, -1, 6) in A belongs to T3.
Point (-1, 6, 6) in A belongs to T4.
Point (4, -3, -6) in A belongs to T5.
Point (-3, 4, -6) in A belongs to T6.
Point (-2, -6, 5) in A belongs to T7.
Point (-6, -4, -3) in A belongs to T8.
Point (6, 5, -2) in A belongs to T2.
Point (6, -1, 6) in A' belongs to T3.
Point (-1, 6, 6) in A' belongs to T4.
Point (4, -3, -6) in A' belongs to T5.
Point (-3, 4, -6) in A' belongs to T6.
Point (-6, -2, 5) in A' belongs to T7.
Point (-4, -6, -3) in A' belongs to T8.
Point (5, 6, -2) in A' belongs to T2.
Point (-6, -4, -3) in P belongs to T8.
Point (-3, -4, -6) in P belongs to T8.
Point (5, -6, -2) in P belongs to T5.
Point (-5, 6, -2) in P belongs to T6.
Point (6, -5, 2) in P belongs to T3.
A, 14-separated
A', 14-separated
